# 1. Nhồi data

In [ ]:
%%bash
git clone https://github.com/hung20204912/cuu_em

In [ ]:
%cd cuu_em

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import GridSearchCV


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
import seaborn as sns


In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
df = pd.read_csv('After Preprocessed.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['sentiment'].value_counts()

In [ ]:
df.isnull().sum()

# 2. Xử lý một vài thứ cho chắc ăn

In [ ]:
import string
punct = string.punctuation
punct

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
stopwords = list(STOP_WORDS)

In [ ]:
def text_data_cleaning(sentence):
  doc = nlp(sentence)

  tokens = [] # list of tokens
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)

  cleaned_tokens = []
  for token in tokens:
    if token not in stopwords and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens

# 3. Train test split

In [ ]:
X = df['review']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train.head()

In [ ]:
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)
classifier = LinearSVC()

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
clf = Pipeline([('tfidf',tfidf), ('clf',classifier)])

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
# confusion_matrix
cf_matrix = confusion_matrix(y_test, y_pred)
categories  = ['Negative','Positive']
group_names = ['True Neg','False Pos', 'False Neg','True Pos']
group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]
labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_names,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
ax = sns.heatmap(cf_matrix, annot = labels,
            fmt='', cmap='Blues',xticklabels = categories, yticklabels = categories)

ax.set_title('Random Forest Confusion Matrix');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

plt.show()

In [ ]:
# classification_report
print(classification_report(y_test, y_pred))

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
clf.predict(["Wow, I am learning Natural Language Processing in fun fashion!"])

In [ ]:
clf.predict(["It's hard to learn new things!"])

In [ ]:
clf.predict(["i love to learn NLP"])

In [ ]:
clf.predict(["my gosh, i don't understand this subject"])